# MA
## Creation of a process for scientific visualization development based on the example of the new ZHAW protein source database
## Christina Köck
## July 2023
### Link to the Gitlab-Repo: https://github.com/TinyTen/MA

Creation of dataframe for iteration 2 with sparkql dataframe and translation of the data into english.

### Libraries and data

In [1]:
# ! pip install cmcrameri

In [2]:
from cmcrameri import cm
import math as math
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import plotly.express as px

import sparql_dataframe

## Read in Dapro Data

In [5]:
endpoint = 'http://localhost:8000'

q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX dapro: <http://dapro.opendata.zhaw.ch/DaPro.owl#>

SELECT ?name
WHERE {
	?dapro a dapro:Product .
    ?dapro rdfs:label ?name.       
}
"""


df_products = sparql_dataframe.get(endpoint, q)
# df_products.head(20)

In [6]:
food_items = df_products['name'].astype('str')
food_items = [str(i) for i in food_items]
food_items.insert(0, 'Apfel')
# food_items

In [7]:
# das funktioniert
d = {}
# food_items = ["Apfel", "Birne", "Ahornsirup", "Brie", "Knoblauch", 'Ackerbohne']

endpoint = 'http://localhost:8000'

for item in food_items:

    q = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX schema: <http://schema.org/>
    PREFIX dapro: <http://dapro.opendata.zhaw.ch/DaPro.owl#>
    SELECT ?foodoningredient ?foodoningredientval ?unit
    WHERE {{
        ?dapro rdfs:label "{}" .
        ?dapro schema:ingredients ?recingredient .
        ?recingredient schema:ingredients ?foodoningredient .
        ?recingredient schema:value ?foodoningredientval .
        ?recingredient schema:unitCode ?unit .   
    }}
        """.format(item)
    
    df_interim = sparql_dataframe.get(endpoint, q)
    df_interim.set_index('foodoningredient', inplace = True)
    df_interim.rename(columns={"foodoningredientval": item}, inplace = True)
    df_interim = df_interim.reindex(sorted(df_interim.index))
    d["df_{}".format(item)] = df_interim
    
df_dapro = d[next(iter(d))]
# # df_dapro.drop('unit', axis = 1, inplace = True)
for key in d.keys():
    if d[key]['unit'].equals(d[next(iter(d))]['unit']):
        df_dapro = df_dapro.merge(d[key][d[key].columns[0]], how='outer', left_index= True,
                               right_on = 'foodoningredient' 
                             )
        

# df_dapro.drop('unit', axis = 1, inplace = True)
df_dapro.set_index('unit', append = True, inplace = True)
df_dapro.index = ['_'.join(ind) for ind in df_dapro.index.values]

df_dapro=df_dapro.T

In [8]:
# read in ecological data:

d = {}


endpoint = 'http://localhost:8000'
for item in food_items:
    q = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX schema: <http://schema.org/>
    PREFIX dapro: <http://dapro.opendata.zhaw.ch/DaPro.owl#>

    SELECT   ?name1 ?UBP
    WHERE {{

        ?ingredient rdfs:label "ecological scarcity 2013, total, UBP/kg in {}"   .
        ?ingredient rdfs:label ?name1.
        ?ingredient schema:value ?UBP .
        OPTIONAL{{ ?ingredient schema:unitCode ?unit .}}   
    }}
    """.format(item)


    df_interim = sparql_dataframe.get(endpoint, q)
    df_interim.set_index('name1', inplace = True)
    df_interim.rename(columns={"name1": item}, inplace = True)
    df_interim = df_interim.reindex(sorted(df_interim.index))
    d["df_{}".format(item)] = df_interim
    
df_UBP = d[next(iter(d))]
# # df_dapro.drop('unit', axis = 1, inplace = True)
for key in d.keys():
#     if d[key]['unit'].equals(d[next(iter(d))]['unit']):
        df_UBP = pd.concat([df_UBP, d[key][d[key].columns[0]]])
        
# df_dapro.drop('unit', axis = 1, inplace = True)
# df_dapro.set_index('unit', append = True, inplace = True)
# df_dapro.index = ['_'.join(ind) for ind in df_dapro.index.values]

# df_dapro=df_dapro.T

In [9]:
df_UBP.drop('UBP', axis = 1, inplace=True)
df_UBP.dropna(axis = 0, inplace = True)

# calculate for 100g not for 1 kg:
df_UBP = df_UBP/10

In [10]:
def lastWord(string):
  # finding the index of last space
  index = string.rfind(" ")
 
  # last word
  return string[index+1:]

In [11]:
actual_food = []
for index in df_UBP.index:
    actual_food.append(lastWord(index))

df_UBP.index = actual_food

In [12]:
df_UBP.rename(columns = {0: 'ecological scarcity 2013, total, UBP/100g'}, inplace=True)
df_dapro.rename(index = {'Apfel_y': 'Apfel'}, inplace=True)
df_dapro.drop(df_dapro.index[0], axis = 0, inplace = True)

In [13]:
df_dapro = df_dapro.merge(df_UBP,  how = 'left', left_index=True, right_index=True)


In [14]:
df_eco = pd.read_excel('UBP.xlsx', sheet_name = 'others')
df_eco.drop('Category', axis = 1, inplace = True)


In [15]:
df_eco.set_index('Food product', inplace = True)

In [16]:
#set pro 100g instead of 1kg:
df_eco = df_eco/10
df_eco.rename(columns = {
    'Eutrophying emissions per kilogram (gPO₄eq per kilogram)': 'Eutrophying emissions per 100g (gPO₄eq per 100g}', 
    'Freshwater withdrawals per kilogram (liters per kilogram)' : 'Freshwater withdrawals per 100g (liters per 100g)',
    'Land use per kilogram (m² per kilogram)': 'Land use per 100g (m² per 100g)'
}
    , inplace = True)

In [17]:
df_eco.rename(index = {'Groundnuts': 'Erdnuss', 
                      'Apples': 'Apfel', 
                      'Milk': 'Kuhmilch',
                      'Cheese': 'Hartkäse',
                      'Eggs': 'Hühnerei'}, inplace=True)

In [18]:
df_dapro = df_dapro.merge(df_eco, how = 'left', left_index=True, right_index=True )

In [19]:
df_dapro.rename(columns = {'Eiweiß (Protein)_mg/100g': 'Eiweiß (Protein)_g/g'
                          }, inplace=True)

In [20]:
# from googletrans import Translator

# translator = Translator()
# columns_en = list(df_dapro.columns[:20].map(lambda x: translator.translate(x, dest='en').text))
# columns_en.append(list(df_dapro.columns[20:40].map(lambda x: translator.translate(x, dest='en').text)))
# print(len(columns_en))
# columns_en.append(list(df_dapro.columns[40:60].map(lambda x: translator.translate(x, dest='en').text)))
# print(len(columns_en))
# columns_en.append(list(df_dapro.columns[60:80].map(lambda x: translator.translate(x, dest='en').text)))
# print(len(columns_en))
# columns_en.append(list(df_dapro.columns[80:100].map(lambda x: translator.translate(x, dest='en').text)))
# print(len(columns_en))
# columns_en.append(list(df_dapro.columns[100:119].map(lambda x: translator.translate(x, dest='en').text)))



In [21]:
from googletrans import Translator

translator = Translator()
columns_en = []

for col in df_dapro.columns:
    word = translator.translate(str(col), dest='en').text
    columns_en.append(word)

IndexError: list index out of range

In [ ]:
len(columns_en)

In [ ]:
columns_en = columns_en + list(df_dapro.columns[119:])

In [ ]:
columns_en

In [ ]:
len(columns_en)

In [ ]:
df_dapro_en = df_dapro
df_dapro_en.columns = columns_en

In [ ]:
df_dapro_en

In [ ]:
from googletrans import Translator

translator = Translator()
index_en = []

for index in df_dapro.index:
    word = translator.translate(str(index), dest='en').text
    index_en.append(word)

In [ ]:
df_dapro_en.index = index_en

In [ ]:
df_dapro_en

In [ ]:
df_dapro_en.rename(index = {'linseed': 'Linseed', 
                            'peanut': 'Peanut', 
                            'hazelnut': 'Hazel nut',
                            'sesame': 'Sesame',
                            'Cashewnuss': 'Cashew nut', 
                            'Pine core': 'Pine nut', 
                           'Trackie': 'Pistachio', 
                           'Sunflower core': 'Sunflower seed', 
                           'Hartweizengriess': 'Durum wheat'}, inplace = True)

In [ ]:
df_dapro_en.rename(columns = {'Protein (protein) _G/g': 'Protein_g/g', 
                            'Energy (kilojoule) _kj/100g': 'Energy (kilojoule) _kJ/100g', 
                            'Natrium_mg/100g': 'Sodium_mg/100g',
                            'Strength_mg/100g': 'Starch_mg/100g',
}, inplace = True)

In [ ]:
df_dapro_en.to_excel('df_dapro_en.xlsx')